In [1]:
import pandas as pd
import sqlite3

In [ ]:
connec = sqlite3.connect("./data/sql-murder-mystery.db")
pd.set_option('display.max_colwidth', None) #para poder ver todo el contenido de los columnas

In [3]:
def read_sql(query):
    
    return pd.read_sql(query, connec)

In [4]:
# SQL

query = """
SELECT name 
  FROM sqlite_master
 where type = 'table'
"""

df = read_sql(query)
df

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [10]:
# escena del crimen
# del enunciado FECHA / TYPE / CITY
query = '''
SELECT * 
FROM crime_scene_report
WHERE date = "20180115" AND type = "murder" AND city = "SQL City"
ORDER BY type
'''
read_sql(query)

# pista: 
# Dos testigos:
# 1º) vive en Northwestern Dr
# 2º) nombre: Annabel y viven en algun sitio en Franklin Ave

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
# person
# Testigo 1
# Filtro por la direccion (y como es la ultiam casa, ordenando)

query = '''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1 -- Limito la busqueda a uno solo
'''
read_sql(query)

# obtengo su ID y su nombre, sería Morty Schapiro

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
# person
# Testigo 2
# Filtro por nombre y direccion

query = '''
SELECT id, name
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'
'''
read_sql(query)

# obtengo su ID y nombre

,id,name
0,16371,Annabel Miller


In [21]:
# interview
# Entrevista del testigo 1

query = '''
SELECT transcript
FROM interview
WHERE person_id = '14887'
'''

read_sql(query)

# Oyó un disparo y vio un hombre salir
# Bolsa con "Get Fit Now Gym" - miembro de oro
# num socio comienza por "48Z"
# Coche matricula con " __H42W__"

,transcript
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [23]:
# interview
# Entrevista del testigo 2

query = '''
SELECT transcript
FROM interview
WHERE person_id = '16371'
'''

read_sql(query)

# Vio el asesinato
# Trabaja en gimnasio
# vio al asesino en el gimnasio el 9th de Enero

,transcript
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [ ]:
'''
PISTAS SOBRE EL ASESINO
- Hombre
- Disparo (pitola)
- Gimnasio: Get Fit Now Gym
- Estuvo en el gimnaso el 9 de Enero
- Num socio comienza por "48Z..."
- Es un miembro de oro del gym
- Coche matricula con " __H42W__"

'''

In [ ]:
# Buscamos psitas al asesino en los datos del gimnasio

In [ ]:
# get_fit_now_member
# de las pistas filtrando en datos del gimnasio
# filtramos por el inicio de su memebership
# miembtro dorado

query = '''
SELECT *
FROM get_fit_now_member
WHERE id LIKE '48Z%' and membership_status="gold"
ORDER BY id
'''

read_sql(query)

# obtenemos dos sospechosos y sus nombres

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


In [ ]:
'''
SOSPECHOSO 1:
    NAME        Jeremy Bowers
    PERSON_ID   67318

SOSPECHOSO 2:
    NAME        Joe Germuska
    PERSON_ID   28819

'''

In [ ]:
# CHECKEO CON LOS DATOS DE ACCESO DEL GIMNASIO
# por si encuentro un sospechoso

# get_fit_now_check_in
# Como sabemos que estuvo el 9 de Enero
# ademas filtramos por el inicio de su memebership

query = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%' AND check_in_date = '20180109'

'''

read_sql(query)

# Seguimos teniendo dos sopechosos

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [ ]:
# Indagamos en las pista del coche

In [ ]:
# drivers_license
# Indagamos por la pista de la matricula del coche
# además de su genero

query = '''
SELECT *
FROM drivers_license
WHERE  plate_number LIKE "%H42W%" AND gender="male"

'''

read_sql(query)

# obtengo dos posibles coches y descripciones

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [61]:
# person
# Busco info de los sopechosos en person

query = '''
SELECT *
FROM person 
WHERE name = 'Joe Germuska' OR name = 'Jeremy Bowers'
'''

read_sql(query)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [ ]:
'''
SOSPECHOSO 1:
    NAME        Jeremy Bowers
    PERSON_ID   67318
    LICENSE_ID  423327
    SSN         871539279

SOSPECHOSO 2:
    NAME        Joe Germuska
    PERSON_ID   28819
    LICENSE_ID  173289
    SSN         138909730
'''

In [ ]:
# drivers_license + person
# cruzo datos de los posibles matriculas de coches y coche-
# con los nombres de los sospechos

query = '''
SELECT p.id, p.name, p.license_id, d.plate_number, d.car_make, d.car_model
FROM person AS p
LEFT JOIN drivers_license AS d 
ON p.license_id = d.id
WHERE p.name = 'Joe Germuska' OR p.name = 'Jeremy Bowers'
'''

read_sql(query)

# hay una coincidencia
# uno de los dos sospechoso parece que no tiene coche

# tenemos al asesino: JEREMY BOWERS

,id,name,license_id,plate_number,car_make,car_model
0,28819,Joe Germuska,173289,None,None,None
1,67318,Jeremy Bowers,423327,0H42W2,Chevrolet,Spark LS


In [ ]:
# interview
# Entrevista del asesino por su ID

query = '''
SELECT *
FROM interview
WHERE person_id = '67318'
'''

read_sql(query)

# Sacamos datos de quien la contrato

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [ ]:
'''
PISTAS DE QUIEN LA CONTRATO

- GENERO    Female
- ALTURA    5'5" (65") / 5'7" (67")
- PELO      Red 
- COCHE     Tesla , Model S
- ESTUVO    SQL Symphony Concert (3 veces)
- FECHA     December 2017

'''

In [149]:
# busco datos en Person + driver_lincese

query = '''
SELECT p.name, p.id, p.ssn, d.height, d.hair_color, d.car_make, d.car_model, d.gender

FROM drivers_license AS d

LEFT JOIN person AS p 
ON d.id = p.license_id

WHERE   d.gender = "female"
        AND (d.height= "65" OR "67")
        AND d.hair_color = "red"
        AND d.car_make = "Tesla"
        AND d.car_model = "Model S"
        
'''

read_sql(query)

# tres sospechosas coinciden

,name,id,ssn,height,hair_color,car_make,car_model,gender
0,Miranda Priestly,99716,987756388,66,red,Tesla,Model S,female
1,Regina George,90700,337169072,66,red,Tesla,Model S,female
2,Red Korb,78881,961388910,65,red,Tesla,Model S,female


In [ ]:
# indago la pista sobre el evento
# filtro primero por la fecha
# añado el nombre del evento
# filtro por los ID de las sospechosas

query = '''
SELECT *
FROM facebook_event_checkin

WHERE   date LIKE "201712%"
        AND event_name LIKE "%SQL%"
        AND (person_id = "78881" OR person_id = "90700" OR person_id = "99716")
'''

read_sql(query)

# solo una persona acudió tres veces, con el ID 99716

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


In [ ]:
# consulto el ID 99716 de nuevo
query = '''
SELECT name
FROM person
WHERE   id = 99716
        
'''

read_sql(query)

# solucion de quién lo organizó

,name
0,Miranda Priestly
